### Barras Book Example: Use British Railway Network to do projections

In [7]:
import os
from dotenv import load_dotenv
import pathlib
from graphdatascience import GraphDataScience

from settings import path_base, path_data
path_to_secrets = pathlib.Path(path_base, 'secrets.env')
load_dotenv(dotenv_path=path_to_secrets)  # Load secrets/env variables


True

##### Connect to the database:

In [8]:
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", os.getenv('NEO4J_PW'))
gds = GraphDataScience(URI, auth=AUTH, database="neo4j")

In [18]:
graphs = ['trains', 'trains_new', 'trains_n']
call = f"""
UNWIND {graphs} AS graph
CALL gds.graph.exists(graph)
YIELD graphName, exists
RETURN graphName, exists
"""

gds.run_cypher(
    query=call
)

for graph in graphs:
    try:
        gds.run_cypher(
        query=f"""
        CALL gds.graph.drop("{graph}") 
        YIELD graphName"""
        )
    except:
        pass

##### Calculate shortest path

In [19]:
gds.graph.project.cypher(
    graph_name="trains",
    node_spec='MATCH (s:Station) RETURN id(s) AS id',
    relationship_spec=
    """
    MATCH (s1:Station)-[t:TRACK]->(s2:Station)
    RETURN id(s1) AS source, id(s2) AS target, t.distance AS distance
    """   
)

GraphCreateResult(graph=Graph({'graphName': 'trains', 'nodeCount': 2593, 'relationshipCount': 5782, 'database': 'neo4j', 'configuration': {'readConcurrency': 4, 'jobId': 'beb883b7-bca0-4eed-90e7-9ee1256d2885', 'logProgress': True, 'relationshipQuery': 'MATCH (s1:Station)-[t:TRACK]->(s2:Station)\n    RETURN id(s1) AS source, id(s2) AS target, t.distance AS distance', 'nodeQuery': 'MATCH (s:Station) RETURN id(s) AS id', 'creationTime': neo4j.time.DateTime(2023, 11, 11, 17, 28, 19, 244002500, tzinfo=<DstTzInfo 'Europe/Berlin' CET+1:00:00 STD>), 'validateRelationships': True, 'parameters': [], 'sudo': True}, 'schema': {'graphProperties': {}, 'nodes': {'__ALL__': {}}, 'relationships': {'__ALL__': {'distance': 'Float (DefaultValue(NaN), TRANSIENT, Aggregation.NONE)'}}}, 'memoryUsage': '7024 KiB'}), result=nodeQuery                         MATCH (s:Station) RETURN id(s) AS id
relationshipQuery    MATCH (s1:Station)-[t:TRACK]->(s2:Station)\n  ...
graphName                                      

##### Shortest path between Birmingham New Street and Edinburgh

In [20]:
birmingham = gds.find_node_id(["Station"], {"name": "Birmingham New Street"})
edinburgh = gds.find_node_id(["Station"], {"name": "Edinburgh"})

In [21]:
shortest_path = gds.shortestPath.dijkstra.stream(
    gds.graph.get("trains"),
    sourceNode = birmingham,
    targetNode = edinburgh,
    relationshipWeightProperty="distance"
)
print(f"Shortest distance: {shortest_path.get('costs').get(0)[-1]}")
gds.close()

Dijkstra:   0%|          | 0/100 [00:00<?, ?%/s]

Shortest distance: 295.91
